# Pratische Umwandlung des Keras-Model zu TensorRT-Model

## Keras-Model zu tensorflow-Model

In [ ]:
import tensorflow as tf
import tensorflow.contrib.tensorrt as trt
import tensorflow.keras as tfk
from tensorflow.python.keras.models import load_model
from tensorflow.compat.v1.graph_util import convert_variables_to_constants
from tensorflow.python.platform import gfile
from tensorflow.python.tools import freeze_graph

### Keras zu Tensorflow-Frozen Model

In [ ]:
keras_model_pfad = ""
tf_model_pfad = ""
fr_graph_tf_model = ""
fr_graph_trt_model = ""
# tfk.backend.set_learning(0) # falls das keras-Model mit Batchnormalisation trainiert wurde
model = load_model =(keras_model_pfad)
saver = tf.train.Saver()
sess = tfk.backend.get_session()
# get input und output des models for tensorflow improvement
input_model = model.inputs[0]
out_model = model.outputs[0]
# save tensort-Graph
tf.saved_model.simple_save(sess,
                          tf_model_pfad,
                          inputs=input_model,
                          outputs=output)
# das model wird jetzt compiliert werden
freeze_graph.freeze_graph(None,
                          None,
                          None,
                          None,
                          model.outputs[0].op.name,
                          None,
                          None,
                          fr_graph_tf_model,
                          False,
                          "",
                          input_saved_model_dir=tf_model_pfad)

### tensorflow-frozen  zu TRT_Frozen-Model(.pb)

In [ ]:
# Erstellung von TRT-Frozen-Model
## read tensorflozen-Graph
with tf.gfile.GFile(fr_graph_tf_model, "rb") as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
trt_graph = trt.create_inference_graph(input_graph_def=tf_frozen_graph,outputs=output_models, max_batch_size=2,max_workspace_size_bytes=2*(10**9), precision_mode="FP32")
# save frozen-graph
with gfile.FastGFile(fr_graph_trt_model, "wb") as f:
    f.write(trt_graph.SerializeToString()))